In [1]:
import json
import os
import glob

In [30]:
fname = glob.glob('../Notebooks/*meta')[0]
print(fname)

../Notebooks/test.meta


In [26]:
with open(fname,'r') as metafile:
    meta_data = json.load(metafile)

In [28]:
meta_data.keys()

dict_keys(['binaryfilename', 'channel_labels', 'data_format', 'experiment_metadata_str', 'fs', 'no_channels', 'no_time_points', 'start_timestamp_unix', 'transmitter_id', 'volts_per_bit'])

In [40]:

params = [
    {'name': 'Metadata', 'type': 'group', 'children':
        [{'name': name, 'type': str(type(meta_data[name]))[8:-2], 'value': meta_data[name]} for name in meta_data.keys()]
     }
]

In [41]:
params[0]['children']

[{'name': 'binaryfilename', 'type': 'str', 'value': 'test.bin'},
 {'name': 'channel_labels',
  'type': 'list',
  'value': ['channel_0', 'channel_1', 'channel_2', 'channel_3']},
 {'name': 'data_format', 'type': 'str', 'value': '>i4'},
 {'name': 'experiment_metadata_str',
  'type': 'str',
  'value': 'kainate_acid_hippo'},
 {'name': 'fs', 'type': 'int', 'value': 512},
 {'name': 'no_channels', 'type': 'int', 'value': 4},
 {'name': 'no_time_points', 'type': 'int', 'value': 1843200},
 {'name': 'start_timestamp_unix', 'type': 'int', 'value': 1564066727},
 {'name': 'transmitter_id', 'type': 'str', 'value': '001'},
 {'name': 'volts_per_bit', 'type': 'float', 'value': 1.940255363782247e-10}]

In [39]:
str(int)[8:-2]

'int'

In [42]:
meta_data

{'binaryfilename': 'test.bin',
 'channel_labels': ['channel_0', 'channel_1', 'channel_2', 'channel_3'],
 'data_format': '>i4',
 'experiment_metadata_str': 'kainate_acid_hippo',
 'fs': 512,
 'no_channels': 4,
 'no_time_points': 1843200,
 'start_timestamp_unix': 1564066727,
 'transmitter_id': '001',
 'volts_per_bit': 1.940255363782247e-10}

In [32]:
import numpy as np


In [40]:
a ={'seizures':np.array([[0,1],[4,5]]),
   'spikes':np.array([[10,20]])}


In [7]:
a.keys()

dict_keys(['seizures', 'spikes'])

In [9]:
'spikes' in a.keys()

True

In [24]:
a['seizures'][0] = []

ValueError: cannot copy sequence with size 0 to array axis with dimension 2

In [17]:
a

{'spikes': [[10, 20]]}

In [16]:
del a['seizures']

In [20]:
[times[0] for times in a['seizures']]

[0, 4]

In [28]:
type(a['seizures'])

numpy.ndarray

In [55]:
a['seizures'].tolist()

[[0, 1], [4, 5]]

In [33]:
json.dumps(a)

'{"seizures": [[0, 1], [4, 5]], "spikes": [[10, 20]]}'

In [42]:
dictionary = a

In [41]:
for element in a:
    print(a[element])

[[0 1]
 [4 5]]
[[10 20]]


In [52]:
shapes = [len(dictionary[label].shape) and dictionary[label].shape[1]==2 for label in dictionary]

In [50]:
all(shapes)

True

In [ ]:
json.load()

In [61]:
from '/home/mfpleite/PycharmProjects/pyecog2/annotations' import annotations

SyntaxError: invalid syntax (<ipython-input-61-96df2879419c>, line 1)

In [2]:
import sys

In [3]:
import json

class Annotations:
    def __init__(self, dictionary=None, fname=None):
        if dictionary is not None and self.check_dict(dictionary):
            self.annotations_dict = dictionary
            self.labels = self.annotations_dict.keys()
        elif fname is not None:
            self.import_from_json(fname)
        else:
            self.annotations_dict = {}
            self.labels = self.annotations_dict.keys()

    @staticmethod
    def check_dict(dictionary):  # try to check if dictionary structure is valid
        if type(dictionary) is not dict:
            return False
        # Now check that all labels have lists of lists of length 2, with numbers inside : TO DO
        if not all([type(dictionary[label]) == list for label in dictionary]):
            print('Annotations: invalid type for annotations dictionary')
            return False
        if not all([all([len(i) == 2 for i in dictionary[label]]) for label in dictionary]):
            print('Annotations: invalid length(s) for [start, end] of annotations')
            return False
        if not all([all([all([type(j) in [int, float] for j in i]) for i in dictionary[label]]) for label in dictionary]):
            print('Annotations: invalid data type for start and end of annotations')
            return False
        return True

    def add_annotation(self, label, tstart, tend):
        if label in self.labels:
            self.annotations_dict[label].append([tstart, tend])
        else:
            self.add_label(label)
            self.annotations_dict[label] = [[tstart, tend]]

    def delete_annotation(self, label, index):
        try:
            del self.annotations_dict[label][index]
        except IndexError:
            print('Annotation index is out of range')

    def delete_label(self, label):
        del self.annotations_dict[label]
        self.labels = self.annotations_dict.keys()

    def add_label(self, label):
        if label not in self.labels:
            self.annotations_dict[label] = []
            self.labels = self.annotations_dict.keys()
        else:
            print('Annotations: Label already exists')

    def set_annotation_times(self, label, index, tstart, tend):
        self.annotations_dict[label][index] = [tstart, tend]

    def get_all_annotation_times(self,label):
        return [i for i in self.annotations_dict[label]]

    def get_all_start_times(self, label):
        return [i[0] for i in self.annotations_dict[label]]

    def get_all_end_times(self, label):
        return [i[1] for i in self.annotations_dict[label]]

    def import_from_json(self, fname):
        with open(fname, 'r') as f:
            self.annotations_dict = json.load(f)
        if not self.check_dict(self.annotations_dict):
            self.annotations_dict = {}
            print('Annotations: invalid dictionary in Json file')
        self.labels = self.annotations_dict.keys()

    def export_to_json(self, fname):
        with open(fname, 'w') as f:
            json.dump(self.annotations_dict, f)

    def export_to_csv(self, fname, label):
        with open(fname, 'w') as f:
            f.write(label + ',' + 'start,stop\n')
            for i,j in enumerate(self.annotations_dict[label]):
                f.write(str(i) + ',' + str(j[0]) + ',' + str(j[1]) + '\n')

    def __repr__(self):
        return repr(self.annotations_dict)

    def __str__(self):
        return str(self.annotations_dict)

In [4]:
a = Annotations({'seizure':[[1,2],[5,6],[5,7]]})

In [5]:
a

{'seizure': [[1, 2], [5, 6], [5, 7], [4, 5.6], [7, 9]]}

In [6]:
a.add_annotation('seizure', 4,5.6)
a.add_annotation('seizure', 7,9)

In [7]:
b = a.get_all_annotation_times('seizure')[0]

In [18]:
b[0]= 0

In [134]:
a.annotations_dict

{'seizure': [[1, 2], [5, 6], [5, 7], [4, 5.6], [7, 9]]}

In [135]:
a

{'seizure': [[1, 2], [5, 6], [5, 7], [4, 5.6], [7, 9]]}

In [136]:
a.delete_annotation('seizure',1)

In [137]:
a.get_all_end_times('seizure')

[2, 7, 5.6, 9]

In [138]:
a.export_to_json('annotations_test.ano')

In [139]:
b = Annotations(fname='annotations_test.ano')

In [140]:
b.annotations_dict

{'seizure': [[1, 2], [5, 7], [4, 5.6], [7, 9]]}

In [141]:
a

{'seizure': [[1, 2], [5, 7], [4, 5.6], [7, 9]]}

In [142]:
a.export_to_csv('annotations_test.csv','seizure')

In [8]:
sum([4,5])

9

In [20]:
3600*500*3*8

43200000

In [21]:
import colorsys

In [30]:
c =array(colorsys.hls_to_rgb(0, .5, .9))*255

NameError: name 'array' is not defined

'{"element": {"element2": 0}}'

In [6]:
d = {'element':{'element2':0}}

'[{"element": {"element2": 0}}]'

In [9]:
a = json.loads(json.dumps([d]))

In [10]:
a

[{'element': {'element2': 0}}]

In [13]:
list(set([1,1,2]))

[1, 2]

In [34]:
a = set(['test','test2'])

{-9, 1, 2}

In [29]:
a

{-9, 1, 2, 4}

In [85]:
class foo:
    def __init__(self,i):
        self.i = i
    def __del__(self):
        print(str(self.i) + ' died')
    def __delete__(self):
        print(str(self.i) + ' died too')

In [110]:
a = [foo(1),foo(2),foo(3)]

In [111]:
a[0]

In [112]:
a

TypeError: list indices must be integers or slices, not list

In [88]:
for i in a:
    del(i)

In [89]:
a

[<__main__.foo at 0x7f0b7f536150>, <__main__.foo at 0x7f0b8c949d90>]

In [95]:
list(b)

In [73]:
b

In [107]:
a = set(['test','test2'])

In [108]:
[s for s in a]

['test', 'test2']

In [223]:
b = [1,3,4,2,3]

In [224]:
for i,e in enumerate(b):
    if e==3:
        del(b[i])
        print(i,e)

1 3
3 3


In [23]:
import json
from PyQt5 import QtCore
from PyQt5.QtCore import QObject
import numpy as np
import colorsys
from collections import OrderedDict

class AnnotationElement(QObject):
    sigAnnotationElementChanged = QtCore.pyqtSignal(object) # emited when any attribute is changed
    sigAnnotationElementDeleted = QtCore.pyqtSignal(object) # emited when all references to annotation are deleted
    def __init__(self, dictionary=None, label='', start=0, end=0, confidence=1, notes=''):
        super().__init__()
        if dictionary is not None:
            header = ['label', 'start', 'end', 'confidence', 'notes']
            self.element_dict = OrderedDict(sorted(dictionary.items(), key=lambda l: header.index(l[0])))
        else:
            # self.element_dict = {'label': label,
            #                      'start': start,
            #                      'end': end,
            #                      'confidence': confidence,
            #                      'notes': notes}  # To be used by classifier predictions
            self.element_dict = OrderedDict([('label', label),
                                             ('start', float(start)),
                                             ('end', float(end)),
                                             ('confidence', float(confidence)),
                                             ('notes', notes)])  # To be used by classifier predictions

    def getKey(self,key):
        return self.element_dict[key]

    def setKey(self,key,value):
        self.element_dict[key] = value
        self.sigAnnotationElementChanged.emit(self)

    def getLabel(self):
        return self.element_dict['label']

    def setLabel(self, start):
        self.element_dict['label'] = start
        self.sigAnnotationElementChanged.emit(self)

    def getStart(self):
        return self.element_dict['start']

    def setStart(self, start):
        self.element_dict['start'] = start
        self.sigAnnotationElementChanged.emit(self)

    def getEnd(self):
        return self.element_dict['end']

    def setEnd(self, end):
        self.element_dict['end'] = end
        self.sigAnnotationElementChanged.emit(self)

    def getPos(self):
        return [self.element_dict['start'], self.element_dict['end']]

    def setPos(self, pos):
        self.element_dict['start'] = min(pos)
        self.element_dict['end'] = max(pos)
        self.sigAnnotationElementChanged.emit(self)

    def getNotes(self):
        return self.element_dict['notes']

    def setNotes(self, notes):
        self.element_dict['notes'] = notes
        self.sigAnnotationElementChanged.emit(self)

    def getConfidence(self):
        return self.element_dict['confidence']

    def setConfidence(self, confidence):
        self.element_dict['confidence'] = confidence
        self.sigAnnotationElementChanged.emit(self)

    def delete(self):
        self.sigAnnotationElementDeleted.emit(self)

    # def __del__(self):
    #      self.sigAnnotationElementDeleted.emit(self)

    def __repr__(self):
        return repr(dict(self.element_dict))

    def __str__(self):
        return str(dict(self.element_dict))

class AnnotationPage(QObject):
    sigFocusOnAnnotation = QtCore.pyqtSignal(object)
    sigAnnotationAdded = QtCore.pyqtSignal(object)

    def __init__(self, alist=None, fname=None):
        super().__init__()
        if list is not None and self.checklist(alist):
            self.annotations_list = alist
            self.labels = list(set([annotation.getLabel() for annotation in alist]))
            self.label_color_dict = {}
            n = max(len(self.labels),6)  # make space for at least 6 colors, or space all colors equally
            for i,label in enumerate(self.labels):
                self.label_color_dict[label] = tuple(np.array(colorsys.hls_to_rgb(i / n, .5, .9)) * 255)

        elif fname is not None:
            self.import_from_json(fname)
        else:
            self.annotations_list = []
            self.labels = []
            self.label_color_dict = {}  # dictionary to save label plotting colors
        self.focused_annotation = None

    def focusOnAnnotation(self, annotation):
        if annotation != self.focused_annotation:
            self.focused_annotation = annotation
            print('focused on', annotation)
            self.sigFocusOnAnnotation.emit(annotation)

    @staticmethod
    def checklist(alist):  # try to check if dictionary structure is valid
        if type(alist) is not list:
            print('Annotations: invalid type for annotations list')
            return False
        # Now check that all elements are of type AnnotationElement
        if not all([isinstance(annotation, AnnotationElement) for annotation in alist]):
            print('Annotations: invalid type for annotations list elements')
            return False
        return True

    def add_annotation(self, annotation):
        self.annotations_list.append(annotation)
        self.add_label(annotation.getLabel())
        self.sigAnnotationAdded.emit(annotation)

    def delete_annotation(self, annotation):
        if type(annotation) is not int:
            index = self.get_annotation_index(annotation)
        else:
            index = annotation # allow for annotation to be the index
            
        try:
            self.annotations_list[index].delete() # send signal we are deleting annotation
            del self.annotations_list[index]
        except IndexError:
            print('Annotation index is out of range')

    def get_annotation_index(self, annotation):
        for i in range(len(self.annotations_list)):
            if self.annotations_list[i] == annotation:
                return i

    def get_all_with_label(self, label):
        result = []
        for i, a in enumerate(self.annotations_list):
            if a.getLabel() == label:
                result.append(a)
        return result

    def delete_all_with_label(self, label):
        for i, a in reversed(list(enumerate(self.annotations_list))):
            if a.getLabel() == label:
                self.delete_annotation(i)

    def add_label(self, label):
        if label not in self.labels:
            self.labels = self.labels.append(label) # for future if labels can have global propreties... probably will never be used
            if label not in self.label_color_dict.keys():
                n = len(self.labels)
                if n <= 6:
                    h = n/6
                else:
                    floorlog = np.floor(np.log(n)/np.log(6))
                    h = 1/(6**floorlog) * ((n - 6**floorlog)*2 + 1)
                self.label_color_dict[label] = tuple(np.array(colorsys.hls_to_rgb(h, .5, .9)) * 255)
        else:
            print('Annotations: Label already exists')

    def export_to_json(self, fname):
        # convert annotation objects into dictionaries
        full_dict = [json.loads(repr(a).replace('\'','\"')) for a in self.annotations_list]
        with open(fname, 'w') as f:
            json.dump([full_dict,list(self.labels),self.label_color_dict], f)

    def import_from_json(self, fname):
        with open(fname, 'r') as f:
            object_list = json.load(f)
        full_dict = object_list[0]
        # convert dictionaries into annotation objects
        self.annotations_list = [AnnotationElement(dictionary=a) for a in full_dict]
        self.labels = set(object_list[1])
        self.label_color_dict = object_list[2]

    def export_to_csv(self, fname, label):
        with open(fname, 'w') as f:
            f.write(label + ',' + 'start,stop\n')
            for i, a in enumerate(self.annotations_list):
                if a.getLabel() == label:
                    f.write(str(i) + ',' + str(a.getStart()) + ',' + str(a.getEnd()) + '\n')

    def __repr__(self):
        return repr([json.loads(repr(a).replace('\'','\"')) for a in self.annotations_list])

    def __str__(self):
        return str([json.loads(repr(a).replace('\'','\"')) for a in self.annotations_list])




In [24]:
annotations = AnnotationPage(alist=[AnnotationElement(label='seizure',start=1,end=10),
                                   AnnotationElement(label='seizure',start=14,end=22),
                                   AnnotationElement(label='spike',start=23,end=25),
                                  ])

In [26]:
annotations.delete_annotation(annotations.annotations_list[2])

In [27]:
annotations

[{'confidence': 1.0, 'end': 10.0, 'start': 1.0, 'label': 'seizure', 'notes': ''}, {'confidence': 1.0, 'end': 22.0, 'start': 14.0, 'label': 'seizure', 'notes': ''}]

In [6]:
s = repr(annotations.annotations_list[0])

In [7]:
repr(AnnotationElement(label='seizure',start=1,end=10))

"{'start': 1, 'label': 'seizure', 'notes': '', 'confidence': 1, 'end': 10}"

In [8]:
annotations.export_to_json('test.txt')

In [9]:
a = AnnotationPage(fname='test.txt')

In [10]:
a.annotations_list[0]

{'start': 1, 'label': 'seizure', 'notes': '', 'confidence': 1, 'end': 10}

In [11]:
a.delete_all_with_label('spike')

In [39]:
a.annotations_list[0].getLabel()

'seizure'

In [40]:
a.label_color_dict

{'seizure': [242.25, 12.75000000000001, 12.75000000000001],
 'spike': [242.24999999999994, 242.25, 12.75000000000001]}

In [60]:
data = [annotation.element_dict for annotation in annotations.annotations_list]

In [62]:
data[0]['label']='spike'

In [67]:
print(data)
print(annotations)

[OrderedDict([('label', 'spike'), ('start', 1), ('end', 10), ('confidence', 1), ('notes', '')]), OrderedDict([('label', 'seizure'), ('start', 14), ('end', 22), ('confidence', 1), ('notes', '')]), OrderedDict([('label', 'spike'), ('start', 23), ('end', 25), ('confidence', 1), ('notes', '')])]
[{'notes': '', 'start': 1, 'end': 10, 'confidence': 1, 'label': 'spike'}, {'notes': '', 'start': 14, 'end': 22, 'confidence': 1, 'label': 'seizure'}, {'notes': '', 'start': 23, 'end': 25, 'confidence': 1, 'label': 'spike'}]


In [288]:
a.get_all_with_label('seizure')[0].setStart(22)

In [289]:
a

[{'label': 'seizure', 'start': 22, 'end': 10, 'notes': '', 'confidence': 1}, {'label': 'seizure', 'start': 14, 'end': 22, 'notes': '', 'confidence': 1}]

In [1]:
asset = set(['a','b','a','c'])

In [7]:
asset

{'a', 'b', 'c'}

In [11]:
col_d = {}

In [12]:
for i,item in enumerate(asset):
    col_d[item] = 0
    print(i,item)

0 a
1 c
2 b


In [13]:
col_d

{'a': 0, 'c': 0, 'b': 0}

In [22]:
json.dumps(list(asset))

'["a", "c", "b"]'

In [2]:
import pyqtgraph.examples

In [3]:
pyqtgraph.examples.run() # check TableWidget, ColorButton and parameter tree

In [3]:
import sys
from PyQt5.QtWidgets import QMainWindow, QApplication, QWidget, QAction, QTableWidget, QTableWidgetItem, QVBoxLayout
from PyQt5.QtGui import QIcon
from PyQt5.QtCore import pyqtSlot


class App(QWidget):

    def __init__(self):
        super().__init__()
        self.title = 'PyQt5 table - pythonspot.com'
        self.left = 0
        self.top = 0
        self.width = 300
        self.height = 200
        self.initUI()

    def initUI(self):
        self.setWindowTitle(self.title)
        self.setGeometry(self.left, self.top, self.width, self.height)

        self.createTable()

        # Add box layout, add table to box layout and add box layout to widget
        self.layout = QVBoxLayout()
        self.layout.addWidget(self.tableWidget)
        self.setLayout(self.layout)

        # Show widget
        self.show()

    def createTable(self):
        # Create table
        self.tableWidget = QTableWidget()
        self.tableWidget.setRowCount(4)
        self.tableWidget.setColumnCount(2)
        self.tableWidget.setItem(0, 0, QTableWidgetItem("Cell (1,1)"))
        self.tableWidget.setItem(0, 1, QTableWidgetItem("Cell (1,2)"))
        self.tableWidget.setItem(1, 0, QTableWidgetItem("Cell (2,1)"))
        self.tableWidget.setItem(1, 1, QTableWidgetItem("Cell (2,2)"))
        self.tableWidget.setItem(2, 0, QTableWidgetItem("Cell (3,1)"))
        self.tableWidget.setItem(2, 1, QTableWidgetItem("Cell (3,2)"))
        self.tableWidget.setItem(3, 0, QTableWidgetItem("Cell (4,1)"))
        self.tableWidget.setItem(3, 1, QTableWidgetItem("Cell (4,2)"))
        self.tableWidget.move(0, 0)

        # table selection change
        self.tableWidget.doubleClicked.connect(self.on_click)

    @pyqtSlot()
    def on_click(self):
        print("\n")
        for currentQTableWidgetItem in self.tableWidget.selectedItems():
            print(currentQTableWidgetItem.row(), currentQTableWidgetItem.column(), currentQTableWidgetItem.text())


if __name__ == '__main__':
    app = QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())  

SystemExit: 0

/home/mfpleite/anaconda3/envs/pyecog_env/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2969: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
from collections import OrderedDict

In [2]:
a = OrderedDict(annotations.annotations_list[0].element_dict)

NameError: name 'annotations' is not defined

In [18]:
dict(a)

{'confidence': 1, 'end': 10, 'label': 'seizure', 'notes': '', 'start': 1}

In [19]:
b = dict(a)

In [42]:
OrderedDict(sorted(b.items(),key=lambda x: ['label','start','end','confidence','notes'].index(x[0])))

OrderedDict([('label', 'seizure'),
             ('start', 1),
             ('end', 10),
             ('confidence', 1),
             ('notes', '')])

In [6]:
from PyQt5 import QtGui, QtWidgets

In [ ]:
QtWidgets.QF

['seizure', 1, 10, 1, '']

In [23]:
b = a.annotations_list[0].getStart()

In [24]:
b

1

In [25]:
a.annotations_list[0].setStart(2)

In [26]:
a

[{'start': 2, 'label': 'seizure', 'notes': '', 'confidence': 1, 'end': 10}, {'start': 14, 'label': 'seizure', 'notes': '', 'confidence': 1, 'end': 22}]

In [27]:
b

1

In [32]:
7&3

3

In [30]:
a = set([2,5,3])

In [29]:
a

[1]

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [31]:
b = list(a)

In [32]:
b

[2, 3, 5]

In [33]:
b[None]

TypeError: list indices must be integers or slices, not NoneType

In [14]:
a = [1]
b=[2]
c=[a,b]

In [36]:
import colorsys

In [256]:
def i_spaced_nfold(i,n):
    floorlog = max(np.floor(np.log2(2*(i-1)/n)),0)
    d   = n*2**(floorlog)
    if d == n:
        h = i-1
    else:
        h = (2*i - d - 1)
    v   = h/d
    val = tuple(np.array(colorsys.hls_to_rgb(v, .5, .9)) * 255)

In [257]:
spaced(1,2)
spaced(2,2)
spaced(3,2)
spaced(4,2)
spaced(5,2)
spaced(8,2)
spaced(9,2)

1 2
floorlog 0 d 2 h 0 v 0.0 val (242.25, 12.750000000000011, 12.750000000000011)
2 2
floorlog 0.0 d 2.0 h 1 v 0.5 val (12.750000000000011, 242.24999999999994, 242.25)
3 2
floorlog 1.0 d 4.0 h 1.0 v 0.25 val (127.50000000000006, 242.25, 12.750000000000011)
4 2
floorlog 1.0 d 4.0 h 3.0 v 0.75 val (127.4999999999999, 12.750000000000011, 242.25)
5 2
floorlog 2.0 d 8.0 h 1.0 v 0.125 val (242.25, 184.875, 12.750000000000011)
8 2
floorlog 2.0 d 8.0 h 7.0 v 0.875 val (242.25, 12.750000000000011, 184.87499999999986)
9 2
floorlog 3.0 d 16.0 h 1.0 v 0.0625 val (242.25, 98.8125, 12.750000000000011)


/home/mfpleite/anaconda3/envs/pyecog_env/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log2
  This is separate from the ipykernel package so we can avoid doing imports until


In [258]:
spaced(1,3)
spaced(2,3)
spaced(3,3)
spaced(4,3)
spaced(5,3)
spaced(8,3)
spaced(9,3)

1 3
floorlog 0 d 3 h 0 v 0.0 val (242.25, 12.750000000000011, 12.750000000000011)
2 3
floorlog 0 d 3 h 1 v 0.3333333333333333 val (12.750000000000011, 242.25, 12.750000000000011)
3 3
floorlog 0.0 d 3.0 h 2 v 0.666666666667 val (12.750000000000011, 12.750000000000011, 242.25)
4 3
floorlog 1.0 d 6.0 h 1.0 v 0.166666666667 val (242.24999999999994, 242.25, 12.750000000000011)
5 3
floorlog 1.0 d 6.0 h 3.0 v 0.5 val (12.750000000000011, 242.24999999999994, 242.25)
8 3
floorlog 2.0 d 12.0 h 3.0 v 0.25 val (127.50000000000006, 242.25, 12.750000000000011)
9 3
floorlog 2.0 d 12.0 h 5.0 v 0.416666666667 val (12.750000000000011, 242.25, 127.50000000000006)


/home/mfpleite/anaconda3/envs/pyecog_env/lib/python3.5/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log2
  This is separate from the ipykernel package so we can avoid doing imports until


In [4]:
a = [1,2,3]
b = [2,a,5]
c = b[1:]     

In [272]:
class a():
    def who(self):
        print('i am a')
    def testa(self):
        print('testa')
        
class b:
    def who(self):
        print('i am b')
    def testb(self):
        print('testb')
        
class c(b,a):
    def testc(self):
        print('testc')

In [8]:
a[0] = 2
c

[[2, 2, 3], 5]

In [275]:
citem.who()

i am b


In [9]:
import time
from datetime import datetime
t =1575664594.3

In [10]:
time.strftime('%H:%M:%S',time.localtime(t))

'21:36:34'

In [14]:
dtt = datetime.fromtimestamp(t)

In [18]:
datetime.strftime(dtt,'%H:%M:%S.%f')

'21:36:34.3000003'

In [1]:
f ="/home/mfpleite/Shared/ele_data/h5_119/M1569833359_2019-09-30-09-49-19_tids_[119].meta"

In [12]:
'.'.join(f.split('.')[:-1]+['features'])

'/home/mfpleite/Shared/ele_data/h5_119/M1569833359_2019-09-30-09-49-19_tids_[119].features'

In [16]:
a = np.arange(1,100,.1)

In [15]:
import numpy as np

In [17]:
a[-1]

99.900000000000091

In [18]:
np.max(np.array([[1,2],[3,4]]))

4

In [19]:
a = [lambda d:np.sum(np.abs(np.diff(d)))]

In [22]:
a[0](np.array([[1,4]]))

3

In [23]:

def rfft_band_power(fdata,fs,band):
    return np.sum(np.abs(fdata[int(len(fdata)*band[0]/fs):int(len(fdata)*band[1]/fs)]))
def powerf(band):
    return lambda fdata,fs: rfft_band_power(fdata,fs,band)

In [25]:
f = powerf((1,3))

In [27]:
f([0,1,2,3,4,5],.1)

0.0